In [0]:
%tensorflow_version 1.x
import sys
sys.path.append('/content/drive/My Drive/Colab_Work/PPM/nn/conv')
sys.path.append('/content/drive/My Drive/Colab_Work/PPM/nn/conv')
sys.path.append('/content/drive/My Drive/Colab_Work/PPM/preprocessing')
sys.path.append('/content/drive/My Drive/Colab_Work/PPM/datasets')
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imagetoarraypreprocessor import ImageToArrayPreprocessor
from aspectawarepreprocessor import AspectAwarePreprocessor
from simplepreprocessor import SimplePreprocessor
from simpledatasetloader import SimpleDatasetLoader
from fcheadnet import FCHeadnet
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import RMSprop
from keras.optimizers import SGD
from keras.applications import VGG16
from keras.layers import Input
from keras.models import Model
from imutils import paths
import numpy as np
import argparse
import os

In [5]:
aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1, 
	    height_shift_range=0.1, shear_range=0.2, zoom_range=0.2, 
	    horizontal_flip=True, fill_mode="nearest")

print("[INFO] loading images...")
%cd /content/drive/My\ Drive/Colab_Work
imagePaths = list(paths.list_images('flowers17'))
classNames = [pt.split(os.path.sep)[-2]for pt in imagePaths]
classNames = [str(x) for x in np.unique(classNames)]

aap = AspectAwarePreprocessor(224, 224)
iap = ImageToArrayPreprocessor()

sdl =SimpleDatasetLoader(preprocessors=[aap, iap])
(data, labels) = sdl.load(imagePaths, verbose=500)
data = data.astype("float")/255.0

(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.25, random_state=42)

trainY = LabelBinarizer().fit_transform(trainY)
testY = LabelBinarizer().fit_transform(testY)

[INFO] loading images...
/content/drive/My Drive/Colab_Work
[INFO] processed 500/1360
[INFO] processed 1000/1360


In [8]:
baseModel = VGG16(weights="imagenet", include_top=False, input_tensor=Input(shape=(224,224,3)))

headModel = FCHeadnet.build(baseModel, len(classNames), 256)

model = Model(inputs=baseModel.input, outputs=headModel)

for layer in baseModel.layers:
	layer.trainable = False

print("[INFO] compiling model...")
opt = RMSprop(lr = 0.001)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

print("[INFO] training model...")
model.fit_generator(aug.flow(trainX, trainY, batch_size=32), 
	validation_data=(testX, testY), epochs=25, 
	steps_per_epoch=len(trainX)//32, verbose=1)

[INFO] compiling model...


[INFO] training model...
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/25
31/31 [==============================] - 16s 515ms/step - loss: 7.3491 - acc: 0.1213 - val_loss: 2.1714 - val_acc: 0.3324
Epoch 2/25
31/31 [==============================] - 10s 337ms/step - loss: 2.3529 - acc: 0.2805 - val_loss: 1.6994 - val_acc: 0.4647
Epoch 3/25
31/31 [==============================] - 10s 329ms/step - loss: 1.9510 - acc: 0.4065 - val_loss: 1.0725 - val_acc: 0.6824
Epoch 4/25
31/31 [==============================] - 10s 326ms/step - loss: 1.6228 - acc: 0.4813 - val_loss: 1.0235 - val_acc: 0.6559
Epoch 5/25
31/31 [==============================] - 10s 327ms/step - loss: 1.4745 - acc: 0.5357 - val_loss: 0.8941 - val_acc: 0.7206
Epoch 6/25
31/31 [==============================] - 10s 327ms/step - loss: 1.3514 - acc: 0.5575 - val_loss: 0.8332 - val_acc: 0.7441
Epoch 7/25
31/31 [==============================] - 1

In [9]:
print("[INFO] evaluating after initialization...")
predictions = model.predict(testX, batch_size=32)
print(classification_report(testY.argmax(axis=1), predictions.argmax(axis=1), target_names=classNames))

for layer in baseModel.layers[15:]:
	layer.trainable = True

[INFO] evaluating after initialization...
              precision    recall  f1-score   support

    bluebell       1.00      0.76      0.86        21
   buttercup       0.94      0.85      0.89        20
  colts_foot       0.94      0.80      0.86        20
     cowslip       0.82      0.90      0.86        20
      crocus       0.52      1.00      0.68        15
    daffodil       0.77      0.87      0.82        23
       daisy       0.96      1.00      0.98        27
   dandelion       0.90      0.95      0.92        19
  fritillary       0.94      0.89      0.92        19
        iris       0.84      1.00      0.91        16
 lily_valley       0.76      1.00      0.86        16
       pansy       1.00      0.67      0.80        18
    snowdrop       0.82      0.90      0.86        20
   sunflower       1.00      0.91      0.95        22
   tigerlily       1.00      0.96      0.98        23
       tulip       0.86      0.67      0.75        18
  windflower       1.00      0.70      

In [10]:
print("[INFO] re-compiling model...")
opt = SGD(lr = 0.001)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

print("[INFO] fine-tuning model...")
model.fit_generator(aug.flow(trainX, trainY, batch_size=32), 
	validation_data=(testX, testY), epochs=25, 
	steps_per_epoch=len(trainX)//32, verbose=1)

[INFO] re-compiling model...
[INFO] fine-tuning model...
Epoch 1/25
31/31 [==============================] - 11s 361ms/step - loss: 0.3970 - acc: 0.8746 - val_loss: 0.3265 - val_acc: 0.9059
Epoch 2/25
31/31 [==============================] - 11s 343ms/step - loss: 0.3804 - acc: 0.8625 - val_loss: 0.3320 - val_acc: 0.8912
Epoch 3/25
31/31 [==============================] - 10s 339ms/step - loss: 0.3179 - acc: 0.8918 - val_loss: 0.3073 - val_acc: 0.9088
Epoch 4/25
31/31 [==============================] - 10s 330ms/step - loss: 0.3113 - acc: 0.8910 - val_loss: 0.3118 - val_acc: 0.8971
Epoch 5/25
31/31 [==============================] - 10s 331ms/step - loss: 0.3014 - acc: 0.9027 - val_loss: 0.3102 - val_acc: 0.9029
Epoch 6/25
31/31 [==============================] - 10s 330ms/step - loss: 0.2687 - acc: 0.9019 - val_loss: 0.3001 - val_acc: 0.9118
Epoch 7/25
31/31 [==============================] - 10s 331ms/step - loss: 0.2812 - acc: 0.9038 - val_loss: 0.3149 - val_acc: 0.9059
Epoch 8/25
3

In [11]:
print("[INFO] evaluating after fine-tuning...")
predictions = model.predict(testX, batch_size=32)
print(classification_report(testY.argmax(axis=1), predictions.argmax(axis=1), target_names=classNames))

print("[INFO] serializing model...")
model.save("flowers17_finetune_model.hdf5")

[INFO] evaluating after fine-tuning...
              precision    recall  f1-score   support

    bluebell       0.90      0.86      0.88        21
   buttercup       0.95      0.95      0.95        20
  colts_foot       0.94      0.80      0.86        20
     cowslip       0.82      0.90      0.86        20
      crocus       0.79      1.00      0.88        15
    daffodil       0.83      0.87      0.85        23
       daisy       1.00      0.96      0.98        27
   dandelion       0.95      0.95      0.95        19
  fritillary       0.95      0.95      0.95        19
        iris       1.00      1.00      1.00        16
 lily_valley       0.94      0.94      0.94        16
       pansy       0.95      1.00      0.97        18
    snowdrop       0.94      0.85      0.89        20
   sunflower       1.00      0.95      0.98        22
   tigerlily       0.96      0.96      0.96        23
       tulip       0.75      0.83      0.79        18
  windflower       0.95      0.87      0.9